In [1]:
# Importing thr SQL and required pakages
import mysql.connector
from tabulate import tabulate

import pandas as pd

import matplotlib.pyplot as plt


# building the connection between MYSQL and PYTHON JUPYTER notebook
connection = mysql.connector.connect(
    host = 'localhost',
    user = 'root',
    password = 'root',
    database = 'hospital_database'
)
# Check the connection is connected before performing the 
if connection.is_connected():
    cursor = connection.cursor()
    print("The connection is established")

def rec_doctorslist():
    print("The Doctors List:")
    
    try:
        query = "select * from doctor"
        cursor.execute(query)
        results = cursor.fetchall()
        
        table = tabulate(results,headers=["did","dname","department"],tablefmt = "grid")
        print(table)
        return table
    
    
    except mysql.connector.Error as err:
        print(f"Mysql Error is: {err}")
    except Exception as e:
        print(f"unknown error is: {e}")
        

def rec_patientlist():
    print("the patients list:")
    
    try:
        query = "select * from patient"
        cursor.execute(query)
        results = cursor.fetchall()
        
        table = tabulate(results,headers =["pid","pname","padd","pmob"],tablefmt="pretty")
        print(table)
        return table
    
    except mysql.connector.Error as err:
        print(f"Mysql Error is: {err}")
    except Exception as e:
        print(f"unknown error is: {e}")
    
def rec_doctorhistory():
    print("the doctor history:")
    
    try:
        dname = input("enter doctor name: ").lower()
        
        # check if the user entered a non-empty string
        if not dname:
            print("Please enter a valid Doctor name.")
            return
        
        show = """
            select a.opdno, a.doa, a.pname ,a.dname, a.symptom, a.treatment
            from appoint a 
            join patient p on a.pname = p.pname 
            where lower(a.dname) like %s
        """
        cursor.execute(show, ('%' + dname + '%',))
        r = cursor.fetchall()
        
        if r:
            print(tabulate(r,headers = ["opdno","doa","pname","dname","symptom","treatment"],tablefmt = "pretty"))
        else:
            print("no record")

    except ValueError as val_err:
        print(f"error is:{val_err}")
        
    except mysql.connector.Error as err:
        print(f"Mysql Error is: {err}")                  
   
    except Exception as e:
        print(f"the unknown error is: {e}")
        
def rec_patienthistory():
    print("Patient History:")

    try:
        pname = input("Enter patient name: ").lower()
        
         # check if the user entered a non-empty string
        if not pname:
            print("Please enter a valid Patient name.")
            return
        

        query = """
            SELECT appoint.opdno, appoint.doa, appoint.pname, appoint.symptom, appoint.treatment, patient.pmob, patient.padd
            FROM appoint
            JOIN patient ON appoint.pname = patient.pname
            WHERE lower(patient.pname) like %s
        """

        cursor.execute(query, ('%' + pname + '%',))
        results = cursor.fetchall()

        if results:
            headers = ["opdno", "doa", "pname", "symptom", "treatment", "pmob", "padd"]
            print(tabulate(results, headers=headers, tablefmt="pretty"))
        else:
            print(f"No records found for patient: {pname}")

    except mysql.connector.Error as err:
        print(f"MySQL Error: {err}")
    except Exception as e:
        print(f"Unknown error: {e}")

        
def rec_opdhistory():
    print("the opd details: ")
    
    try:
        beginning_date = input("enter the beginning date in YYYY-MM-DD: ")
        
        end_date = input("enter the ending date in YYYY-MM-DD: ")
        
        # validate the date format
        if not (beginning_date and end_date):
            print("invalid date format. please use YYYY-MM-DD.")
            
            return
        
        query = "select a.opdno,a.doa,a.pname,a.dname,a.symptom,a.treatment,o.fee from appoint a join opd o on a.opdno = o.opdno where a.doa between %s and %s"
        
        cursor.execute(query,(beginning_date, end_date))
        
        results = cursor.fetchall()
        
        if results:
            print(tabulate(results,headers = ["opdno","doa","pname","dname","symptom","treatment","fee"], tablefmt = "pretty"))
            
        else:
            print("no record")

    except mysql.connector.Error as err:
        print(f"Mysql Error is: {err}")
        
    except Exception as e:
        print(f"unknown error is : {e}")
        

############################ Diagnose chart #####################################################


def rec_diagnosechart():
    
    print("maximum Diagnose: ")
    
    try:
        query = "select symptom, count(symptom) as total_cases from appoint group by symptom;"
         
        df = pd.read_sql(query,connection)
        
        if not df.empty:
            print(tabulate(df, headers = "keys",tablefmt = "pretty"))
            
        
            plt.bar(df.symptom, df.total_cases, color = "green")
            
            plt.xlabel("symptom")
            plt.ylabel("total cases")
            
            plt.title("Diagnose Chart")
            
            plt.show()   
        else:
            print("no record")
        
    except ValueError as val_err:
        print(f"Invalid input:{val_err}")
        
    except mysql.connector.Error as err:
        print(f"Mysql Error is: {err}")
        
    except Exception as e:
        print(f"unknown error is : {e}")   
        


The connection is established
